## 라이브러리 import

In [1]:
import calendar  # .monthrange(year, month) : 해당년도, 해당월의 날짜수
from bs4 import BeautifulSoup  # html 분석

from selenium import webdriver as wd  # 크롤링 웹 접속
from selenium.webdriver.common.by import By

import urllib
from urllib.request import urlretrieve
import urllib.parse
from urllib.request import urlopen

import requests

import time
from tqdm import tqdm

import pandas as pd

## 데이터 수집이 필요한 구, 동 list 생성

In [2]:
# 연립 매매
MultiHouse_sales = {"중구" : ["을지로4가" ,"을지로5가", "주교동", "방산동", "을지로3가", "입정동", "산림동", "초동",
                            "인현동1가", "저동2가"],
                    "용산구" : ["이촌동"], "도봉구" : ["창동"], "노원구" : ["중계동"], "양천구" : ["신정동"],
                    "강서구" : ["가양동"], "강동구" : ["둔촌동"]}


# 연립 전월세
MultiHouse_rent = {"노원구" : ["중계동"], "양천구" : ["신정동"], "강서구" : ["가양동"], "강동구" : ["둔촌동"]}
MultiHouse_rent2 = {"강서구" : ["가양동"], "강동구" : ["둔촌동"]}

# 연립빌라_매매 결측치 수집

 - 서울부동산정보광장 <br>
 - http://land.seoul.go.kr/land/rtms/rtmsMultiHouse.do

### 연립빌라 매매 데이터 수집 & 데이터프레임 저장 (함수)

In [3]:
def villa_sales(gu, dong, result):
    # 데이터 프레임 준비
    df_tmp = pd.DataFrame(columns = ["구", "법정동", "지번", "연립/빌라이름", "면적", "매매가격"])
    
    for i in range(len(result)):
        # \n으로 한 번 나누기
        result_tmp = result[i].text.split('\n')
        
        # 1) 빌라 이름이 있을 때
        if result_tmp[0][0] != " ":
            
            # 건물명
            villa_name = result_tmp[0]
            
            # 지번 
            villa_jibun = result_tmp[1].split(" ")[1].strip()
            
            # 면적
            villa_squre = result_tmp[1].split(" ")[2].strip()
        
        # 2) 빌라 이름이 없을 때
        else:
            
            # 건물명 - 앞에서 따온 걸로 채우기
            result_tmp.insert(0, villa_name)
            
            # 지번 - 앞에서 따온 걸로 채우기
            result_tmp.insert(1, villa_jibun)
            
            # 면적
            villa_squre = result_tmp[2].split("  ")[2]
        
        
        
        # 가격
        price_list = " ".join(result_tmp[1:]).split(" ")
        # 한 셀에 여러개 이을 경우를 대비해서 리스트 준비
        price_tmp = []
        
        for price in price_list:
            # (층) 정보 삭제
            if ',' in price:
                price_tmp.append(price.split("(")[0])
                
                for villa_price in price_tmp:
                    print(villa_price)
                    
                    print("번호:", i, "구:", gu, " 동:", dong, " 지번:", villa_jibun, " 연립/빌라이름:", villa_name,
                          " 면적:", villa_squre, " 매매가격:", villa_price)
                    df_tmp = df_tmp.append({"구" : gu, "법정동" : dong, "지번" : villa_jibun, "연립/빌라이름" : villa_name,
                                            "면적" : villa_squre, "매매가격" : villa_price}, ignore_index=True)
    
    return df_tmp

In [93]:
# 반복문 구조 check
for gu in apt_rent.keys():
    print(gu)
    dong_list = apt_rent[gu]
    
    for dong in dong_list:
        print(dong)
        
        for i in range(1,5):
            print(i)

종로구
연지동
1
2
3
4
효제동
1
2
3
4
종로5가
1
2
3
4
종로6가
1
2
3
4
충신동
1
2
3
4
중구
을지로4가
1
2
3
4
을지로5가
1
2
3
4
주교동
1
2
3
4
방산동
1
2
3
4
을지로3가
1
2
3
4
입정동
1
2
3
4
산림동
1
2
3
4
초동
1
2
3
4
인현동1가
1
2
3
4
저동2가
1
2
3
4
중랑구
면목동
1
2
3
4
도봉구
창동
1
2
3
4
노원구
하계동
1
2
3
4
상계동
1
2
3
4
양천구
신정동
1
2
3
4
강서구
가양동
1
2
3
4
구로구
구로동
1
2
3
4
서초구
반포동
1
2
3
4
강남구
개포동
1
2
3
4
대치동
1
2
3
4
일원동
1
2
3
4
수서동
1
2
3
4
송파구
문정동
1
2
3
4
장지동
1
2
3
4
신천동
1
2
3
4
강동구
둔촌동
1
2
3
4
상일동
1
2
3
4
강일동
1
2
3
4


### TEST

In [4]:
driver = wd.Chrome(executable_path="chrome_driver/chromedriver.exe")
driver.implicitly_wait(3)

apt_url = "http://land.seoul.go.kr/land/rtms/rtmsMultiHouse.do" 
driver.get(apt_url)
page = urlopen(apt_url)                              # urlopen 사용

In [7]:
# 구분선택
driver.find_element_by_id("selectJRGubun").send_keys("지번")
time.sleep(1) 
driver.find_element_by_id("selectSigungu").send_keys("강동구")
time.sleep(1)
driver.find_element_by_id("selectBjdong").send_keys("둔촌동")
time.sleep(1)
driver.find_element_by_id("selectGubun").send_keys("분기")
time.sleep(1)
driver.find_element_by_id("selectYear").send_keys("2019")
time.sleep(1)
driver.find_element_by_id("selectBoongi").send_keys("1분기")
time.sleep(1)
# driver.find_element_by_id("selectJWGubun").send_keys("전세")
# time.sleep(1)


# 조회 버튼 클릭
xp = """//*[@id="search"]"""
driver.find_element_by_xpath(xp).click()
time.sleep(2) 

result_raw = driver.find_element_by_id("resultList")
result = result_raw.find_elements_by_tag_name("tr")

WebDriverException: Message: chrome not reachable
  (Session info: chrome=86.0.4240.75)


In [6]:
# 함수 test
df_new = villa_sales("강동구", "둔촌동", result)

18,000
번호: 0 구: 강동구  동: 둔촌동  지번: 442-7  연립/빌라이름: 그린캐슬  면적: 38.79  매매가격: 18,000
40,000
번호: 1 구: 강동구  동: 둔촌동  지번: 442  연립/빌라이름: 금강트레조빌  면적: 52.51  매매가격: 40,000
20,700
번호: 2 구: 강동구  동: 둔촌동  지번: 470-5  연립/빌라이름: 동서빌라  면적: 33.95  매매가격: 20,700
23,000
번호: 3 구: 강동구  동: 둔촌동  지번: 147-11  연립/빌라이름: 런던빌  면적: 40.71  매매가격: 23,000
23,000
번호: 4 구: 강동구  동: 둔촌동  지번: 147-11  연립/빌라이름: 런던빌  면적: 40.74  매매가격: 23,000
32,000
번호: 5 구: 강동구  동: 둔촌동  지번: 110-1  연립/빌라이름: 명문아이빌1단지  면적: 46.25  매매가격: 32,000
11,000
번호: 6 구: 강동구  동: 둔촌동  지번: 98-66  연립/빌라이름: 선영하우스  면적: 12.2  매매가격: 11,000
11,000
번호: 6 구: 강동구  동: 둔촌동  지번: 98-66  연립/빌라이름: 선영하우스  면적: 12.2  매매가격: 11,000
11,000
번호: 6 구: 강동구  동: 둔촌동  지번: 98-66  연립/빌라이름: 선영하우스  면적: 12.2  매매가격: 11,000
11,000
번호: 6 구: 강동구  동: 둔촌동  지번: 98-66  연립/빌라이름: 선영하우스  면적: 12.2  매매가격: 11,000
11,000
번호: 6 구: 강동구  동: 둔촌동  지번: 98-66  연립/빌라이름: 선영하우스  면적: 12.2  매매가격: 11,000
11,000
번호: 6 구: 강동구  동: 둔촌동  지번: 98-66  연립/빌라이름: 선영하우스  면적: 12.2  매매가격: 11,000
11,000
번호: 7 구: 강동구  동: 둔촌동  지번: 98-66  연립/빌라이름:

In [831]:
# 확인1
len(result)

88

In [832]:
# 확인2
df_new

,구,법정동,지번,연립/빌라이름,면적,매매가격
0,도봉구,창동,296-34,(296-34),54.88,"13,500"
1,도봉구,창동,581-51,(581-51),50.08,"16,000"
2,도봉구,창동,585-92,(585-92),68.4,"14,400"
3,도봉구,창동,553-41,5차장수타운,59.88,"12,400"
4,도봉구,창동,667-11,가현월디움,52.02,"31,500"
...,...,...,...,...,...,...
98,도봉구,창동,645-12,한신빌리지12동,33.42,"6,300"
99,도봉구,창동,582-66,한일주택,38.28,"10,800"
100,도봉구,창동,603-48,현대그린맨션,63.44,"15,800"
101,도봉구,창동,461-13,혜성루첸리(461-13),45.82,"23,800"


## 실제 수집

In [8]:
# 데이터 프레임 준비
df_villaSales_2019 = pd.DataFrame(columns = ["구", "법정동", "지번", "연립/빌라이름", "면적", "매매가격"])


# 크롬 드라이버 준비
driver = wd.Chrome(executable_path="chrome_driver/chromedriver.exe")
driver.implicitly_wait(3)

# 연립 매매 URL로 연결
apt_url = "http://land.seoul.go.kr/land/rtms/rtmsMultiHouse.do" 
driver.get(apt_url)
page = urlopen(apt_url)                              # urlopen 사용


# 매매 tab 클릭
xp = """//*[@id="content"]/div[3]/div[1]/section/ul/li[1]/a"""
driver.find_element_by_xpath(xp).click()
time.sleep(3) 


# 연립/빌라 매매 정보 수집 시작
for gu in tqdm(MultiHouse_sales.keys()):
    
    # 지역선택 - 지번 선택
    driver.find_element_by_id("selectJRGubun").send_keys("지번")
    time.sleep(3)
    
    # 지역선택 - 구 선택
    driver.find_element_by_id("selectSigungu").send_keys(gu)
    time.sleep(3)
    dong_list = MultiHouse_sales[gu]
    
    
    for dong in tqdm(dong_list):
        # 지역선택 - 동 선택
        driver.find_element_by_id("selectBjdong").send_keys(dong)
        time.sleep(3)
        
        # 기간선택 - 2019년
        driver.find_element_by_id("selectGubun").send_keys("분기")
        time.sleep(3)
        driver.find_element_by_id("selectYear").send_keys("2019")
        time.sleep(3)
        
        # 기간선택 -1,2,3,4분기
        for i in range(1,5):
            driver.find_element_by_id("selectBoongi").send_keys(str(i)+"분기")
            time.sleep(3)
            
            
            # 조회버튼 - click
            xp = """//*[@id="search"]"""
            driver.find_element_by_xpath(xp).click()
            time.sleep(2) 
            
            
            # 데이터 - table 전부 가져오기
            result_raw = driver.find_element_by_id("resultList")
               
            # 데이터 - row별로 가져오기
            result = result_raw.find_elements_by_tag_name("tr")
            
            if result[0].text == '조회된 내용이 없습니다.':
                continue
            
            
            # 함수 - 가져온 정보를 컬럼별로 구분 한 후 데이터프레임에 넣어서 return
            new_df = villa_sales(gu, dong, result)
            
            # 이미 만들어진 df_villaSales_2019 데이터프레임에 concat 계속 하기
            df_villaSales_2019 = pd.concat([df_villaSales_2019, new_df], ignore_index=True)



  0%|                                                                                            | 0/7 [00:00<?, ?it/s]


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]


 10%|████████▎                                                                          | 1/10 [00:29<04:25, 29.53s/it]


 20%|████████████████▌                                                                  | 2/10 [00:59<03:56, 29.54s/it]


 30%|████████████████████████▉                                                          | 3/10 [01:28<03:26, 29.53s/it]


 40%|█████████████████████████████████▏                                                 | 4/10 [01:58<02:57, 29.53s/it]


 50%|█████████████████████████████████████████▌                                         | 5/10 [02:27<02:27, 29.53s/it]


 60%|█████████████████████████████████████████████████▊                                 | 6/10 [02:57<01:58, 29.52s/it]


 70%|█████████████████

60,000
번호: 0 구: 용산구  동: 이촌동  지번: 203-33  연립/빌라이름: (203-33)  면적: 46.88  매매가격: 60,000
60,000
번호: 1 구: 용산구  동: 이촌동  지번: 203-39  연립/빌라이름: (203-39)  면적: 26.98  매매가격: 60,000
65,000
번호: 0 구: 용산구  동: 이촌동  지번: 203-39  연립/빌라이름: (203-39)  면적: 26.98  매매가격: 65,000
60,000
번호: 1 구: 용산구  동: 이촌동  지번: 199-26  연립/빌라이름: 삼우(3동)  면적: 47.07  매매가격: 60,000
66,000
번호: 0 구: 용산구  동: 이촌동  지번: 203-39  연립/빌라이름: (203-39)  면적: 26.98  매매가격: 66,000
63,000
번호: 0 구: 용산구  동: 이촌동  지번: 199-14  연립/빌라이름: (199-14)  면적: 46.81  매매가격: 63,000
55,000
번호: 1 구: 용산구  동: 이촌동  지번: 199-8  연립/빌라이름: (199-8)  면적: 62.48  매매가격: 55,000
84,500
번호: 2 구: 용산구  동: 이촌동  지번: 203-25  연립/빌라이름: (203-25)  면적: 77.13  매매가격: 84,500
69,000
번호: 3 구: 용산구  동: 이촌동  지번: 203-27  연립/빌라이름: (203-27)  면적: 66.39  매매가격: 69,000
65,000
번호: 4 구: 용산구  동: 이촌동  지번: 203-39  연립/빌라이름: (203-39)  면적: 26.98  매매가격: 65,000
65,000
번호: 4 구: 용산구  동: 이촌동  지번: 203-39  연립/빌라이름: (203-39)  면적: 26.98  매매가격: 65,000
64,900
번호: 4 구: 용산구  동: 이촌동  지번: 203-39  연립/빌라이름: (203-39)  면적: 26.98  매매가격: 64,




100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:29<00:00, 29.86s/it]


 29%|███████████████████████▋                                                           | 2/7 [05:37<18:28, 221.66s/it]


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

13,500
번호: 0 구: 도봉구  동: 창동  지번: 296-34  연립/빌라이름: (296-34)  면적: 54.88  매매가격: 13,500
16,000
번호: 1 구: 도봉구  동: 창동  지번: 581-51  연립/빌라이름: (581-51)  면적: 50.08  매매가격: 16,000
14,400
번호: 2 구: 도봉구  동: 창동  지번: 585-92  연립/빌라이름: (585-92)  면적: 68.4  매매가격: 14,400
12,400
번호: 3 구: 도봉구  동: 창동  지번: 553-41  연립/빌라이름: 5차장수타운  면적: 59.88  매매가격: 12,400
31,500
번호: 4 구: 도봉구  동: 창동  지번: 667-11  연립/빌라이름: 가현월디움  면적: 52.02  매매가격: 31,500
7,250
번호: 5 구: 도봉구  동: 창동  지번: 609-147  연립/빌라이름: 거성빌라  면적: 47.25  매매가격: 7,250
20,000
번호: 6 구: 도봉구  동: 창동  지번: 462-17  연립/빌라이름: 그린빌라  면적: 40.56  매매가격: 20,000
14,000
번호: 7 구: 도봉구  동: 창동  지번: 691-24  연립/빌라이름: 금성빌라24  면적: 56.75  매매가격: 14,000
17,000
번호: 8 구: 도봉구  동: 창동  지번: 691-8  연립/빌라이름: 금성빌라8  면적: 54.11  매매가격: 17,000
15,300
번호: 9 구: 도봉구  동: 창동  지번: 663-33  연립/빌라이름: 금용맨션  면적: 44.91  매매가격: 15,300
11,000
번호: 10 구: 도봉구  동: 창동  지번: 609-136  연립/빌라이름: 금호빌라(609-136)  면적: 38.61  매매가격: 11,000
21,700
번호: 11 구: 도봉구  동: 창동  지번: 521-1  연립/빌라이름: 다성홈타운(521-1)  면적: 43.69  매매가격: 21,700
23,950
번호: 12 구: 도

번호: 84 구: 도봉구  동: 창동  지번: 582-66  연립/빌라이름: 한일주택  면적: 38.28  매매가격: 10,800


...




100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:45<00:00, 45.03s/it]


 71%|███████████████████████████████████████████████████████████▎                       | 5/7 [07:53<03:32, 106.20s/it]


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

24,500
번호: 0 구: 강서구  동: 가양동  지번: 239-20  연립/빌라이름: 그랜드빌  면적: 46.91  매매가격: 24,500
25,000
번호: 1 구: 강서구  동: 가양동  지번: 239-21  연립/빌라이름: 우신빌라트2차  면적: 59.56  매매가격: 25,000
25,900
번호: 2 구: 강서구  동: 가양동  지번: 238-7  연립/빌라이름: 파크빌리지  면적: 56.09  매매가격: 25,900
25,900
번호: 2 구: 강서구  동: 가양동  지번: 238-7  연립/빌라이름: 파크빌리지  면적: 56.09  매매가격: 25,900
25,000
번호: 2 구: 강서구  동: 가양동  지번: 238-7  연립/빌라이름: 파크빌리지  면적: 56.09  매매가격: 25,000
24,000
번호: 3 구: 강서구  동: 가양동  지번: 237-3  연립/빌라이름: 한림빌라  면적: 55.95  매매가격: 24,000
33,500
번호: 0 구: 강서구  동: 가양동  지번: 249-6  연립/빌라이름: 홍림아트빌나동  면적: 73.32  매매가격: 33,500
23,000
번호: 1 구: 강서구  동: 가양동  지번: 236-2  연립/빌라이름: 흥림아트빌1차  면적: 74.67  매매가격: 23,000
23,500
번호: 0 구: 강서구  동: 가양동  지번: 239-20  연립/빌라이름: 그랜드빌  면적: 43.43  매매가격: 23,500
25,500
번호: 1 구: 강서구  동: 가양동  지번: 151-11  연립/빌라이름: 상아맨션2차  면적: 51.17  매매가격: 25,500
31,000
번호: 2 구: 강서구  동: 가양동  지번: 253  연립/빌라이름: 우신빌라트  면적: 74.67  매매가격: 31,000
25,100
번호: 3 구: 강서구  동: 가양동  지번: 238-3  연립/빌라이름: 청솔펠리체  면적: 45.49  매매가격: 25,100
25,300
번호: 4 구: 강서구  동: 가양동  지번: 2




100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:29<00:00, 29.91s/it]


 86%|████████████████████████████████████████████████████████████████████████            | 6/7 [08:29<01:25, 85.14s/it]


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

18,000
번호: 0 구: 강동구  동: 둔촌동  지번: 442-7  연립/빌라이름: 그린캐슬  면적: 38.79  매매가격: 18,000
40,000
번호: 1 구: 강동구  동: 둔촌동  지번: 442  연립/빌라이름: 금강트레조빌  면적: 52.51  매매가격: 40,000
20,700
번호: 2 구: 강동구  동: 둔촌동  지번: 470-5  연립/빌라이름: 동서빌라  면적: 33.95  매매가격: 20,700
23,000
번호: 3 구: 강동구  동: 둔촌동  지번: 147-11  연립/빌라이름: 런던빌  면적: 40.71  매매가격: 23,000
23,000
번호: 4 구: 강동구  동: 둔촌동  지번: 147-11  연립/빌라이름: 런던빌  면적: 40.74  매매가격: 23,000
32,000
번호: 5 구: 강동구  동: 둔촌동  지번: 110-1  연립/빌라이름: 명문아이빌1단지  면적: 46.25  매매가격: 32,000
11,000
번호: 6 구: 강동구  동: 둔촌동  지번: 98-66  연립/빌라이름: 선영하우스  면적: 12.2  매매가격: 11,000
11,000
번호: 6 구: 강동구  동: 둔촌동  지번: 98-66  연립/빌라이름: 선영하우스  면적: 12.2  매매가격: 11,000
11,000
번호: 6 구: 강동구  동: 둔촌동  지번: 98-66  연립/빌라이름: 선영하우스  면적: 12.2  매매가격: 11,000
11,000
번호: 6 구: 강동구  동: 둔촌동  지번: 98-66  연립/빌라이름: 선영하우스  면적: 12.2  매매가격: 11,000
11,000
번호: 6 구: 강동구  동: 둔촌동  지번: 98-66  연립/빌라이름: 선영하우스  면적: 12.2  매매가격: 11,000
11,000
번호: 6 구: 강동구  동: 둔촌동  지번: 98-66  연립/빌라이름: 선영하우스  면적: 12.2  매매가격: 11,000
11,000
번호: 7 구: 강동구  동: 둔촌동  지번: 98-66  연립/빌라이름:




100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:32<00:00, 32.25s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [09:07<00:00, 78.21s/it]


In [9]:
# 확인
df_villaSales_2019

,구,법정동,지번,연립/빌라이름,면적,매매가격
0,용산구,이촌동,203-33,(203-33),46.88,"60,000"
1,용산구,이촌동,203-39,(203-39),26.98,"60,000"
2,용산구,이촌동,203-39,(203-39),26.98,"65,000"
3,용산구,이촌동,199-26,삼우(3동),47.07,"60,000"
4,용산구,이촌동,203-39,(203-39),26.98,"66,000"
...,...,...,...,...,...,...
1419,강동구,둔촌동,600-3,정산빌라,50.04,"21,800"
1420,강동구,둔촌동,45,진흥연립주택,66.36,"30,500"
1421,강동구,둔촌동,46-57,천우빌라1차,53.87,"21,600"
1422,강동구,둔촌동,605-14,한강빌라,53.17,"22,500"


In [10]:
# 저장
df_villaSales_2019.to_csv("data_csv/연립빌라_매매_결측치수집_지번주소(cp949).csv", encoding="cp949", index = False)

In [6]:
driver.quit()

# 2) 연립빌라_전세 결측치 수집

 - 서울부동산정보광장<br>
 - http://land.seoul.go.kr/land/rtms/rtmsMultiHouse.do

### 연립빌라 전세 데이터 수집 & 데이터프레임 저장 (함수)

In [14]:
def villa_jeonse(gu, dong, result):
    # 데이터 프레임 준비
    df_tmp = pd.DataFrame(columns = ["구", "법정동", "지번", "연립/빌라이름", "면적", "전세보증금"])
    
    for i in range(len(result)):
        # \n으로 한 번 나누기
        result_tmp = result[i].text.split('\n')
    
        # 만약 첫번째 칸이 비어있으면, 바로 위의 값으로 채워야 함.
        # 그 값을 받는 if 문 : 칸이 비어있지 않으면 각 위치의 값을 변수에 저장해두기
        if result_tmp[0][0] != " ":

            # 건물명
            villa_name = result_tmp[0]

            # 지번 
            villa_jibun = result_tmp[1].split(" ")[1].strip()

            # 면적
            villa_squre = result_tmp[1].split(" ")[2].strip()

        # 2) 빌라 이름이 없을 때
        else:

            # 건물명 - 앞에서 따온 걸로 채우기
            result_tmp.insert(0, villa_name)

            # 지번 - 앞에서 따온 걸로 채우기
            result_tmp.insert(1, villa_jibun)

            # 면적
            villa_squre = result_tmp[2].split("  ")[2]
            
            
        # 가격 구하기
        price_list = " ".join(result_tmp[1:]).split(" ")
        
        for villa_price in price_list:
            if ',' in villa_price:
                villa_price = villa_price.split("(")[0]
                
                # 확인용
                print("번호 :", i, "건물이름 :", villa_name, "/  지번 :", villa_jibun, 
                      "/  면적 :",villa_squre, "/  가격 :",villa_price)
                
                # 데이터프레임에 업데이트
                df_tmp = df_tmp.append({"구" : gu, "법정동" : dong, "지번" : villa_jibun, "연립/빌라이름" : villa_name,
                                        "면적" : villa_squre, "전세보증금" : villa_price}, ignore_index=True)
                
    # 최종 데이터프레임 return
    return df_tmp

### TEST

In [5]:
driver = wd.Chrome(executable_path="chrome_driver/chromedriver.exe")
driver.implicitly_wait(3)

apt_url = "http://land.seoul.go.kr/land/rtms/rtmsMultiHouse.do" 
driver.get(apt_url)
page = urlopen(apt_url)                              # urlopen 사용


# 구분선택

xp1 = """//*[@id="content"]/div[3]/div[1]/section/ul/li[2]/a"""
driver.find_element_by_xpath(xp1).click()

time.sleep(2) 
driver.find_element_by_id("selectJRGubun").send_keys("지번")
time.sleep(1) 
driver.find_element_by_id("selectSigungu").send_keys("도봉구")
time.sleep(1)
driver.find_element_by_id("selectBjdong").send_keys("창동")
time.sleep(1)
driver.find_element_by_id("selectGubun").send_keys("분기")
time.sleep(1)
driver.find_element_by_id("selectYear").send_keys("2019")
time.sleep(1)
driver.find_element_by_id("selectBoongi").send_keys("1분기")
time.sleep(1)
driver.find_element_by_id("selectJWGubun").send_keys("전세")
time.sleep(1)


# 조회 버튼 클릭
xp = """//*[@id="search"]"""
driver.find_element_by_xpath(xp).click()
time.sleep(2) 

result_raw = driver.find_element_by_id("resultList")
result1 = result_raw.find_elements_by_tag_name("tr")

In [ ]:
driver.close()

In [52]:
# 확인1
len(result1)

121

In [53]:
# 확인2
df_check = villa_jeonse("도봉구", "창동", result1)
df_check

번호 : 0 건물이름 : (484-22) /  지번 : 484-22 /  면적 : 34.59 /  가격 : 8,000
번호 : 1 건물이름 : (519-20) /  지번 : 519-20 /  면적 : 31.4 /  가격 : 5,000
번호 : 2 건물이름 : 5차장수타운 /  지번 : 553-41 /  면적 : 59.03 /  가격 : 18,000
번호 : 3 건물이름 : JINI-vill /  지번 : 715-9 /  면적 : 38.49 /  가격 : 15,000
번호 : 4 건물이름 : space+5 /  지번 : 263-5 /  면적 : 35.14 /  가격 : 14,000
번호 : 5 건물이름 : 가나빌라B동 /  지번 : 533-32 /  면적 : 41.49 /  가격 : 13,500
번호 : 6 건물이름 : 가현월디움 /  지번 : 667-11 /  면적 : 52.02 /  가격 : 30,000
번호 : 6 건물이름 : 가현월디움 /  지번 : 667-11 /  면적 : 52.02 /  가격 : 28,900
번호 : 6 건물이름 : 가현월디움 /  지번 : 667-11 /  면적 : 52.02 /  가격 : 36,000
번호 : 7 건물이름 : 고탑빌라101 /  지번 : 632-78 /  면적 : 107.38 /  가격 : 20,000
번호 : 8 건물이름 : 광신주택(601-80) /  지번 : 601-80 /  면적 : 33.06 /  가격 : 7,400
번호 : 9 건물이름 : 금성빌라19 /  지번 : 691-19 /  면적 : 39.18 /  가격 : 9,500
번호 : 10 건물이름 : 금성파크빌3 /  지번 : 665-24 /  면적 : 45.94 /  가격 : 15,000
번호 : 11 건물이름 : 낙원그랑빌B /  지번 : 746-83 /  면적 : 66.69 /  가격 : 16,000
번호 : 11 건물이름 : 낙원그랑빌B /  지번 : 746-83 /  면적 : 66.69 /  가격 : 16,000
번호 : 12 건물이름 : 내

,구,법정동,지번,연립/빌라이름,면적,전세보증금
0,도봉구,창동,484-22,(484-22),34.59,"8,000"
1,도봉구,창동,519-20,(519-20),31.4,"5,000"
2,도봉구,창동,553-41,5차장수타운,59.03,"18,000"
3,도봉구,창동,715-9,JINI-vill,38.49,"15,000"
4,도봉구,창동,263-5,space+5,35.14,"14,000"
...,...,...,...,...,...,...
137,도봉구,창동,607-29,호암그린맨션,35.97,"7,000"
138,도봉구,창동,607-29,호암그린맨션,51.8,"7,500"
139,도봉구,창동,560-6,화이트빌,28,"10,000"
140,도봉구,창동,442-7,효승하이빌,54.72,"17,000"


## 실제 수집

In [20]:
# 셀레니움 드라이버 연결
driver = wd.Chrome(executable_path="chrome_driver/chromedriver.exe")
driver.implicitly_wait(3)

# URL 접속
apt_url = "http://land.seoul.go.kr/land/rtms/rtmsMultiHouse.do" 
driver.get(apt_url)
page = urlopen(apt_url)                              # urlopen 사용

# 전월세 tab 클릭
xp1 = """//*[@id="content"]/div[3]/div[1]/section/ul/li[2]/a"""
driver.find_element_by_xpath(xp1).click()

# 데이터 프레임 준비
df_villaJoense_2019 = pd.DataFrame(columns = ["구", "법정동", "지번", "연립/빌라이름", "면적", "전세보증금"])



# 아파트 전월세
for gu in tqdm(MultiHouse_rent.keys()):
    
    # 지역선택 - 지번 선택
    driver.find_element_by_id("selectJRGubun").send_keys("지번")
    time.sleep(3)
    
    # 지역선택 - 구 선택
    driver.find_element_by_id("selectSigungu").send_keys(gu)
    time.sleep(3)
    dong_list = MultiHouse_rent[gu]
    
    for dong in tqdm(dong_list):
        # 지역선택 - 동 선택
        driver.find_element_by_id("selectBjdong").send_keys(dong)
        time.sleep(3)
        
        # 기간선택 - 2019년
        driver.find_element_by_id("selectGubun").send_keys("분기")
        time.sleep(3)
        driver.find_element_by_id("selectYear").send_keys("2019")
        time.sleep(3)        
        
        # 기간선택 -1,2,3,4분기
        for i in range(1,5):
            driver.find_element_by_id("selectBoongi").send_keys(str(i)+"분기")
            time.sleep(3)
            
            # 전세 선택
            driver.find_element_by_id("selectJWGubun").send_keys("전세")
            time.sleep(2) 
            
            # 조회버튼 - click
            xp1 = """//*[@id="search"]"""
            driver.find_element_by_xpath(xp1).click()
            time.sleep(2) 
            
            # 데이터 - table 전부 가져오기
            result_raw = driver.find_element_by_id("resultList")
               
            # 데이터 - row별로 가져오기
            result = result_raw.find_elements_by_tag_name("tr")
            
            if result[0].text == '조회된 내용이 없습니다.':
                continue
            
            # 데이터 - 가공한 후 데이터프레임에 넣기
            new_df = villa_jeonse(gu, dong, result)
            
            # 수집된 데이터를 df_aptSales_2019 뒤로 계속 붙여넣기
            df_villaJoense_2019 = pd.concat([df_villaJoense_2019, new_df], ignore_index=True)








  0%|                                                                                            | 0/4 [00:00<?, ?it/s]







  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

번호 : 0 건물이름 : (90-27)쉬즈빌 /  지번 : 90-27 /  면적 : 44.24 /  가격 : 8,500
번호 : 1 건물이름 : Mean.house(75-45) /  지번 : 75-45 /  면적 : 52.1 /  가격 : 17,300
번호 : 2 건물이름 : 궁전빌라 /  지번 : 75-20 /  면적 : 55.41 /  가격 : 8,000
번호 : 3 건물이름 : 궁전빌라 /  지번 : 75-20 /  면적 : 57.92 /  가격 : 10,500
번호 : 3 건물이름 : 궁전빌라 /  지번 : 75-20 /  면적 : 57.92 /  가격 : 10,500
번호 : 4 건물이름 : 도암빌라(75-15) /  지번 : 75-15 /  면적 : 41.76 /  가격 : 10,000
번호 : 5 건물이름 : 부강그린빌라트C동 /  지번 : 109-9 /  면적 : 44.7 /  가격 : 10,000
번호 : 5 건물이름 : 부강그린빌라트C동 /  지번 : 109-9 /  면적 : 44.7 /  가격 : 10,000
번호 : 6 건물이름 : 부강쉘브르C /  지번 : 90-75 /  면적 : 57.21 /  가격 : 16,300
번호 : 7 건물이름 : 산장아트빌라(90-34) /  지번 : 90-34 /  면적 : 49.58 /  가격 : 10,000
번호 : 8 건물이름 : 상명타운 /  지번 : 62-7 /  면적 : 57.99 /  가격 : 16,000
번호 : 9 건물이름 : 선화 /  지번 : 107-25 /  면적 : 46.44 /  가격 : 11,500
번호 : 10 건물이름 : 세한빌라 /  지번 : 68-5 /  면적 : 54.48 /  가격 : 17,000
번호 : 11 건물이름 : 숲속파크빌(90-24) /  지번 : 90-24 /  면적 : 71.76 /  가격 : 19,000
번호 : 12 건물이름 : 쉬즈빌2차 /  지번 : 107-49 /  면적 : 42.56 /  가격 : 15,000
번호 : 13 건물이름 : 신성빌









100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:40<00:00, 40.07s/it]







 25%|█████████████████████                                                               | 1/4 [00:46<02:18, 46.17s/it]







  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

번호 : 0 건물이름 : (1002-10) /  지번 : 1002-10 /  면적 : 51.36 /  가격 : 17,000
번호 : 1 건물이름 : (1002-2) /  지번 : 1002-2 /  면적 : 52.44 /  가격 : 10,000
번호 : 2 건물이름 : (177-24) /  지번 : 177-24 /  면적 : 40.01 /  가격 : 11,000
번호 : 3 건물이름 : (177-24) /  지번 : 177-24 /  면적 : 52.16 /  가격 : 14,000
번호 : 4 건물이름 : (177-24) /  지번 : 177-24 /  면적 : 59.64 /  가격 : 15,500
번호 : 5 건물이름 : (177-71) /  지번 : 177-71 /  면적 : 48.94 /  가격 : 14,500
번호 : 6 건물이름 : (888-34) /  지번 : 888-34 /  면적 : 65.28 /  가격 : 16,000
번호 : 6 건물이름 : (888-34) /  지번 : 888-34 /  면적 : 65.28 /  가격 : 16,000
번호 : 7 건물이름 : (960-11) /  지번 : 960-11 /  면적 : 43.88 /  가격 : 13,000
번호 : 8 건물이름 : (991-13) /  지번 : 991-13 /  면적 : 45.27 /  가격 : 11,000
번호 : 9 건물이름 : (998-14) /  지번 : 998-14 /  면적 : 46.86 /  가격 : 17,800
번호 : 10 건물이름 : (998-14) /  지번 : 998-14 /  면적 : 48.48 /  가격 : 16,000
번호 : 11 건물이름 : G.S Park B동 /  지번 : 952-9 /  면적 : 24.3 /  가격 : 18,000
번호 : 11 건물이름 : G.S Park B동 /  지번 : 952-9 /  면적 : 24.3 /  가격 : 18,500
번호 : 12 건물이름 : G.S Park B동 /  지번 : 952-9 /  면적 : 25.71 

번호 : 29 건물이름 : 뉴파인빌 /  지번 : 924-27 /  면적 : 52.97 /  가격 : 17,000
번호 : 30 건물이름 : 다나팰리스 /  지번 : 935-3 /  면적 : 41.7 /  가격 : 15,000
번호 : 31 건물이름 : 다다빌라트(973-22) /  지번 : 973-22 /  면적 : 53.57 /  가격 : 18,000
번호 : 32 건물이름 : 다다빌라트(973-22) /  지번 : 973-22 /  면적 : 96.46 /  가격 : 25,000
번호 : 33 건물이름 : 다다빌라트(다동) /  지번 : 1212-7 /  면적 : 77.91 /  가격 : 10,000
번호 : 34 건물이름 : 다솔빌라 /  지번 : 999-9 /  면적 : 72.8 /  가격 : 25,500
번호 : 35 건물이름 : 다우빌라B동 /  지번 : 876-17 /  면적 : 84.42 /  가격 : 18,000
번호 : 36 건물이름 : 다우빌라C동 /  지번 : 879-15 /  면적 : 64.56 /  가격 : 17,000
번호 : 37 건물이름 : 대신빌라 /  지번 : 1194-1 /  면적 : 45.66 /  가격 : 10,000
번호 : 38 건물이름 : 대영 /  지번 : 939-9 /  면적 : 35.22 /  가격 : 15,000
번호 : 39 건물이름 : 더채움하우스 /  지번 : 940-14 /  면적 : 25.28 /  가격 : 18,500
번호 : 40 건물이름 : 더채움하우스 /  지번 : 940-14 /  면적 : 26.26 /  가격 : 19,000
번호 : 41 건물이름 : 더테라스(944-6) /  지번 : 944-6 /  면적 : 40.78 /  가격 : 16,000
번호 : 42 건물이름 : 더테라스1 /  지번 : 963-32 /  면적 : 40.29 /  가격 : 17,000
번호 : 43 건물이름 : 더테라스4 /  지번 : 975-22 /  면적 : 45.33 /  가격 : 23,000
번호 : 44

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



번호 : 715 건물이름 : 주풍빌리지 /  지번 : 622-9 /  면적 : 79.81 /  가격 : 32,000
번호 : 716 건물이름 : 주홍빌라트 /  지번 : 484 /  면적 : 58.76 /  가격 : 22,000
번호 : 717 건물이름 : 주황아트빌 /  지번 : 489-13 /  면적 : 35.29 /  가격 : 16,000
번호 : 718 건물이름 : 중앙하우스 /  지번 : 209-3 /  면적 : 49.82 /  가격 : 32,000
번호 : 719 건물이름 : 지에스하우징허브 /  지번 : 289-43 /  면적 : 26.31 /  가격 : 21,000
번호 : 720 건물이름 : 지에스하우징허브 /  지번 : 289-43 /  면적 : 29.93 /  가격 : 22,000
번호 : 721 건물이름 : 지하우스 /  지번 : 87-1 /  면적 : 30.95 /  가격 : 21,000
번호 : 722 건물이름 : 지하우스 /  지번 : 87-1 /  면적 : 50.44 /  가격 : 29,000
번호 : 723 건물이름 : 지하우스 /  지번 : 87-1 /  면적 : 53.18 /  가격 : 32,010
번호 : 724 건물이름 : 지하우스 /  지번 : 87-1 /  면적 : 56.01 /  가격 : 31,000
번호 : 725 건물이름 : 진성드림빌 /  지번 : 430-3 /  면적 : 47.63 /  가격 : 20,500
번호 : 726 건물이름 : 집앤사 비디아파트 /  지번 : 545-7 /  면적 : 20.83 /  가격 : 19,500
번호 : 726 건물이름 : 집앤사 비디아파트 /  지번 : 545-7 /  면적 : 20.83 /  가격 : 21,000
번호 : 726 건물이름 : 집앤사 비디아파트 /  지번 : 545-7 /  면적 : 20.83 /  가격 : 19,500
번호 : 726 건물이름 : 집앤사 비디아파트 /  지번 : 545-7 /  면적 : 20.83 /  가격 : 19,500
번호 : 727 건

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 26,000
번호 : 845 건물이름 : 한영캐슬 /  지번 : 457-1 /  면적 : 43.34 /  가격 : 34,000
번호 : 846 건물이름 : 한영펠리스 /  지번 : 442-1 /  면적 : 34.31 /  가격 : 31,000
번호 : 847 건물이름 : 한영펠리스 /  지번 : 442-1 /  면적 : 36.9 /  가격 : 31,000
번호 : 847 건물이름 : 한영펠리스 /  지번 : 442-1 /  면적 : 36.9 /  가격 : 31,000
번호 : 847 건물이름 : 한영펠리스 /  지번 : 442-1 /  면적 : 36.9 /  가격 : 30,000
번호 : 847 건물이름 : 한영펠리스 /  지번 : 442-1 /  면적 : 36.9 /  가격 : 31,000
번호 : 848 건물이름 : 한영펠리스 /  지번 : 442-1 /  면적 : 39.71 /  가격 : 30,000
번호 : 848 건물이름 : 한영펠리스 /  지번 : 442-1 /  면적 : 39.71 /  가격 : 30,000
번호 : 848 건물이름 : 한영펠리스 /  지번 : 442-1 /  면적 : 39.71 /  가격 : 32,000
번호 : 849 건물이름 : 한우리빌 /  지번 : 492-24 /  면적 : 58.18 /  가격 : 18,000
번호 : 850 건물이름 : 한울글로리 /  지번 : 482-3 /  면적 : 50.1 /  가격 : 17,000
번호 : 851 건물이름 : 해성빌라 /  지번 : 231-28 /  면적 : 39.36 /  가격 : 26,000
번호 : 852 건물이름 : 해승빌 /  지번 : 437-33 /  면적 : 51.51 /  가격 : 20,000
번호 : 853 건물이름 : 해오름빌 /  지번 : 438-40 /  면적 : 29.83 /  가격 : 24,000
번호 : 854 건물이름 : 해인하우스11차 /  지번 : 219-9 /  면적 : 28.6 /  가격 : 29,000
번호 : 855 건물이름 : 해인하우스1









100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [02:10<00:00, 130.74s/it]







100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [04:50<00:00, 72.71s/it]


In [21]:
df_villaJoense_2019

,구,법정동,지번,연립/빌라이름,면적,전세보증금
0,노원구,중계동,90-27,(90-27)쉬즈빌,44.24,"8,500"
1,노원구,중계동,75-45,Mean.house(75-45),52.1,"17,300"
2,노원구,중계동,75-20,궁전빌라,55.41,"8,000"
3,노원구,중계동,75-20,궁전빌라,57.92,"10,500"
4,노원구,중계동,75-20,궁전빌라,57.92,"10,500"
...,...,...,...,...,...,...
5462,강동구,둔촌동,481-10,효성쉐르빌,75.5,"19,000"
5463,강동구,둔촌동,382-4,훼미리빌라,73.15,"13,000"
5464,강동구,둔촌동,516-19,"휴먼리버빌(A,B,C동)",72.26,"27,300"
5465,강동구,둔촌동,434-9,흥진로지스빌,18.55,"7,500"


In [22]:
# 저장
df_villaJoense_2019.to_csv("data_csv/연립빌라_전세_결측치수집_지번주소(cp949).csv", encoding="cp949", index = False)

In [23]:
driver.quit()

# 2) 연립빌라_월세 결측치 수집

 - 서울부동산정보광장<br>
 - http://land.seoul.go.kr/land/rtms/rtmsMultiHouse.do